# La fonction `dual`

La fonction `dual` telle qu'implémentée dans JuMP traduit une notion légèrement différente de la dualité, tel qu'illustré ci-dessous.

In [1]:
using JuMP
using LinearAlgebra
using HiGHS

Considérons le programme linéaire suivant.

In [2]:
m = Model()

@variable(m, x[1:3])

@constraint(m, c1, sum(x[i] for i = 1:2) >= 3)
@constraint(m, c2, -2x[1] + 2x[2] - 4x[3] <= 5)

@constraint(m, n2, x[2] >= 0)
@constraint(m, n3, x[3] <= 0)

@objective(m, Min, 4x[1]+2x[2]+x[3])

println(m)

Min 4 x[1] + 2 x[2] + x[3]
Subject to
 c1 : x[1] + x[2] >= 3.0
 n2 : x[2] >= 0.0
 c2 : -2 x[1] + 2 x[2] - 4 x[3] <= 5.0
 n3 : x[3] <= 0.0



In [3]:
set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

Presolving model
2 rows, 3 cols, 5 nonzeros
1 rows, 2 cols, 2 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-3); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  6.5000000000e+00
HiGHS run time      :          0.00


In [4]:
value.(x)

3-element Vector{Float64}:
  0.25
  2.75
 -0.0

Nous pouvons utiliser la fonction `dual` pour directement obtenir les valeurs des variables duales, mais comme expliqué à la page https://jump.dev/JuMP.jl/stable/manual/constraints/, la fonction objectif n'est pas prise en compte comme le concept utilisé est celui de la dualité conique: https://jump.dev/MathOptInterface.jl/v1.9/background/duality/#Duality

Dans le cas d'un programme de minimisation, nous obtenons le résultats souhaité.

In [5]:
[dual(c1) ; dual(c2)]

2-element Vector{Float64}:
  3.0
 -0.5

JuMP propose aussi la fonction `shadow_price` qui évalue les changements dans l'objectif si nous relâchons le contrainte d'une unité. Le terme relaxation n'est pas précisément défini, mais nous pouvons voir que lorsque nous avons une inégalité plus grand que, le signe est l'opposé de ce que nous attendions.

In [6]:
[shadow_price(c1) ; shadow_price(c2)]

2-element Vector{Float64}:
 -3.0
 -0.5

Formons explicitement le problème dual afin de valider nos observations.

In [7]:
m = Model()

@variable(m, y[1:2])

@constraint(m, c1, y[1] - 2y[2] == 4)
@constraint(m, c2, y[1] + 2y[2] <= 2)
@constraint(m, c3, -4y[2] >= 1)

@constraint(m, n1, y[1] >= 0)
@constraint(m, n2, y[2] <= 0)

@objective(m, Max, 3y[1]+5y[2])

println(m)

Max 3 y[1] + 5 y[2]
Subject to
 c1 : y[1] - 2 y[2] == 4.0
 c3 : -4 y[2] >= 1.0
 n1 : y[1] >= 0.0
 c2 : y[1] + 2 y[2] <= 2.0
 n2 : y[2] <= 0.0



In [8]:
set_optimizer(m, HiGHS.Optimizer)

optimize!(m)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-5); columns 0(-2); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  6.5000000000e+00
HiGHS run time      :          0.00


In [9]:
value.(y)

2-element Vector{Float64}:
  3.0
 -0.5

Le problème est maintenant un programme linéaire de maximisation, et quand nous appelons la fonction `dual`, nous obtenons l'opposé la solution primale optimale.

In [10]:
[ dual(c1) ; dual(c2) ; dual(c3) ]

3-element Vector{Float64}:
 -0.25
 -2.75
  0.0

Ici, les "shadow prices" correspondent à ce que nous attendions.

In [11]:
[ shadow_price(c1) ; shadow_price(c2) ; shadow_price(c3) ]

3-element Vector{Float64}:
 0.25
 2.75
 0.0

Nous pouvons aussi observer ce comportement à partir de l'exemple donné à l'adresse https://jump.dev/JuMP.jl/stable/manual/constraints/

In [12]:
model = Model(HiGHS.Optimizer)
@variable(model, x)
@constraint(model, con, x <= 1)
@objective(model, Min, -2x)
optimize!(model)
dual(con)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-1); elements 0(-1) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -2.0000000000e+00
HiGHS run time      :          0.00


-2.0

In [13]:
shadow_price(con)

-2.0

In [14]:
 @objective(model, Max, 2x)

2 x

In [15]:
optimize!(model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  2.0000000000e+00
HiGHS run time      :          0.00


In [16]:
dual(con)

-2.0

In [17]:
shadow_price(con)

2.0

In [18]:
model = Model(HiGHS.Optimizer)
@variable(model, x)
@constraint(model, con, -x >= 1)
@objective(model, Min, -2x)
optimize!(model)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-1); elements 0(-1) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.0000000000e+00
HiGHS run time      :          0.00


In [19]:
dual(con)

2.0

In [20]:
shadow_price(con)

-2.0

In [21]:
@objective(model, Max, 2x)
optimize!(model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     : -2.0000000000e+00
HiGHS run time      :          0.00


In [22]:
dual(con)

2.0

In [23]:
shadow_price(con)

2.0